In [1]:
!pip install pandas numpy matplotlib seaborn nltk tqdm datasets scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 47.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 56.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 60.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 92.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from tqdm import tqdm
from datasets import load_dataset

### Load dataset

In [6]:
from huggingface_hub import login

login()

In [7]:
# Load dataset
dataset = load_dataset("chcaa/periphery-aviser-e5", split='train')

df = dataset.to_pandas()
df.shape

README.md:   0%|          | 0.00/758 [00:00<?, ?B/s]

train-00000-of-00013.parquet:   0%|          | 0.00/369M [00:00<?, ?B/s]

train-00001-of-00013.parquet:   0%|          | 0.00/369M [00:00<?, ?B/s]

train-00002-of-00013.parquet:   0%|          | 0.00/373M [00:00<?, ?B/s]

train-00003-of-00013.parquet:   0%|          | 0.00/388M [00:00<?, ?B/s]

...

train-00010-of-00013.parquet:   0%|          | 0.00/369M [00:00<?, ?B/s]

train-00011-of-00013.parquet:   0%|          | 0.00/373M [00:00<?, ?B/s]

train-00012-of-00013.parquet:   0%|          | 0.00/384M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/685021 [00:00<?, ? examples/s]

(685021, 13)

### Annotated books

In [5]:
books = pd.read_csv('../../newspaper_temp_files/random_sample_250409.csv', index_col=0)
books.head()

,article_id,date,book_announce,comment,text,118,77
264863,ode_023787,1814-09-30,y,NaN,Paa Adressekontoiret faaes : Figurer til Tegne...,NaN,NaN
151177,aal_066100,1828-05-19,n,mixed,"Hos Bogtrykker Wandall, boendes paa Hjørnet af...",NaN,NaN
379019,ode_138229,1839-07-05,n,NaN,"Placat. Fredagen den førstkommende 26 Julii, F...",NaN,NaN
483748,thi_040569,1839-11-14,n,mixed,J disse Dage er paa Boghandler Høst's Forlag u...,NaN,NaN
369240,ode_128401,1837-11-13,n,NaN,Fra Rentekammeret er under 5 October udkommet ...,NaN,NaN


In [ ]:
books = books[['article_id', 'book_announce', 'comment']]
books = books[books['book_announce'].isin(['y', 'n'])]
books.shape

In [16]:
feuills = pd.read_csv('../../newspaper_temp_files/feuilleton_annotation_250409.csv')
feuills.head()

,article_id,date,is_book_announcement,is_feuilleton,feuilleton_id,feuilleton_author,text
0,thi_000004,1824-05-06,n,bio,thi_napoleon-skildring,Philips,Den store engelske Taler Phillips Carakteer-Sk...
1,thi_000005,1824-05-06,n,bio,thi_napoleon-skildring,Philips,"Stor, skummel, og egen sad han paa Thronen, li..."
2,thi_000006,1824-05-06,n,bio,thi_napoleon-skildring,Philips,"En fast, kjæk og selvstændig Sjæl, en Villie d..."
3,thi_000007,1824-05-06,n,bio,thi_napoleon-skildring,Philips,"Kastet ind i Verden i Midten af en Revolution,..."
4,thi_000008,1824-05-06,n,bio,thi_napoleon-skildring,Philips,Med ingen anden Ven end sit Sværd og ingen i F...


In [17]:
feuills = feuills[['article_id', 'is_book_announcement']]
feuills = feuills[feuills['is_book_announcement'].isin(['y'])]
feuills.shape

(70, 2)

In [19]:
books['book_announce'].value_counts()

book_announce
n    472
y    118
Name: count, dtype: int64

In [20]:
feuills['article_id'].isin(books['article_id']).value_counts()

article_id
False    70
Name: count, dtype: int64

In [24]:
feuills = feuills.rename(columns={'is_book_announcement': 'book_announce'})
feuills.head()

,article_id,book_announce
166,thi_000483,y
282,thi_001221,y
337,thi_001409,y
425,thi_002470b,y
537,thi_005082,y


In [27]:
book_gold = pd.concat([books, feuills])

In [28]:
book_gold['book_announce'].value_counts()

book_announce
n    472
y    188
Name: count, dtype: int64

In [30]:
df = df.merge(book_gold, on='article_id')